# Python

### Open Source Workshop Pandas

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
df = pd.read_csv('Kernindicatoren.csv', delimiter=',', header='infer', index_col=None, skiprows=0, skip_blank_lines=True, on_bad_lines='warn')

In [ ]:
df.head(5)

In [ ]:
df.columns

In [ ]:
df['waarde'].tail(10)

In [ ]:
df.info()

In [ ]:
df['Periode '] = pd.to_datetime(df['Periode '],format='%Y-%m',errors='coerce')
df.head(5)

In [ ]:
df.info()

In [ ]:
xls_file = pd.ExcelFile('kernindicatoren.xlsx')
df = pd.read_excel(xls_file,sheet_name='Data', usecols='A, B, C, D, E') 
df['Periode'] = pd.to_datetime(df['Periode'],format='%Y-%m',errors='coerce')
df.head(10)

### Data schonen 

In [ ]:
df.info()

In [ ]:
df['waarde'] = pd.to_numeric(df['waarde'], errors='coerce')
df.isna().sum()

In [ ]:
df.dropna(how='any', thresh=2, axis=0, subset=['waarde','Instrument'], inplace=False).head(10)

In [ ]:
df_interpolated = df[['Instrument', 'waarde']].interpolate(method='linear', order=2, limit=5, limit_direction='both', limit_area='inside')
df_interpolated.head(10)

In [ ]:
df.loc[df['waarde'].isna(), 'waarde'] = df_interpolated['waarde'].loc[df['waarde'].isna()]

In [ ]:
print("Aantal rijen voor verwijderen {}".format(len(df)))
df.drop_duplicates(inplace=True)
print("Aantal rijen na verwijderen {}".format(len(df)))

In [ ]:
df['waarde - 10'] = df['waarde'] - 10 
df.head(5)

In [ ]:
df.drop(columns=['waarde - 10'], inplace=True)
df.head(5)

In [ ]:
df.round({'waarde' : -2}).head(5)

In [ ]:
df.loc[df['Periode']=='2006-06']

In [ ]:
df.iloc[45:55,:3]

In [ ]:
df.loc[df['Periode']=='2006-06'].loc[:,'waarde']

In [ ]:
df.loc[(df['Instrument'].str.contains('hypotheken', case=False, na=False)) 
       & (df['InstrumentSub'].str.contains('zuiver nieuwe leningen', case=False, na=False)) 
       & (df['InstrumentSub'].str.contains('rente', case=False, na=False)) 
       & ((df['waarde'] >= 3) | (df['waarde'] < 1.7))]

In [ ]:
df2 = df[df['Periode']<'2004-01-01'].set_index(['Instrument','InstrumentSub','Sector','Periode'],append=False)
df2.head(20)

In [ ]:
df2.loc['Consumptief krediet',:,'Huishoudens',: ]

In [ ]:
df2 = df.set_index('Periode').head(20).reindex(pd.date_range('10/1/2002', periods=240, freq='MS'))
df2

In [ ]:
df1 = df.tail(10)
df2 = df.head(10)
pd.concat([df1, df2])

In [ ]:
raw_data_a = {
        'account_id': ['0', '1', '2', '3', '4'],
        'datum': ['2009-12', '2010-01', '2009-12', '2010-01', '2010-01'],
        'type': ['Consumptief krediet', 'Woninghypotheken', 'Zakelijke kredietverlening', 'Woninghypotheken', 'Zakelijke kredietverlening'],
        'initieel bedrag': [5218, 268000, 30000, 415000, 235400]}
df_a = pd.DataFrame(raw_data_a, columns = ['account_id', 'datum', 'type', 'initieel bedrag'])
display(df_a)
print('')
raw_data_b = {
        'account_id': ['0', '0', '1', '3', '0', '2', '1', '3', '3', '2','5'],
        'datum': ['2009-12', '2010-01', '2010-01', '2010-01', '2010-01', '2010-01', '2010-02', '2010-02', '2010-02', '2010-02', '2010-02'],
        'afbetaling': [211, 516, 245, 189, 520, 110, 548, 328, 1065, 985, 1050],
        'type': ['standaard', 'standaard', 'standaard', 'standaard', 'extra', 'standaard', 'standaard', 'standaard', 'extra', 'standaard', 'standaard']}
df_b = pd.DataFrame(raw_data_b, columns = ['account_id', 'datum', 'afbetaling','type'])
display(df_b)

In [ ]:
pd.merge(df_a, df_b, how='outer', left_on=['account_id', 'datum'], right_on=['account_id', 'datum'],suffixes=('_a', '_b'),indicator=True)

In [ ]:
raw_data_a = {
        'account_id': ['0', '1', '2', '3', '4'],
        'waarde': [5218, 268000, 30000, 415000, 235400],
        'aantal': [1, 1, 2, 2, 1]}
df_a = pd.DataFrame(raw_data_a, columns = ['waarde', 'aantal'], index = raw_data_a['account_id'])
display(df_a)
print('')
raw_data_b = {
        'account_id': ['1', '0', '3', '2'],
        'waarde': [100, 200, 300, 400],
        'aantal': [1, 1, 1, 1]}
df_b = pd.DataFrame(raw_data_b, columns = ['waarde', 'aantal'], index = raw_data_b['account_id'])
display(df_b)
print('')
display(df_a+df_b)

In [ ]:
df.sort_values(by=['Periode','Sector'], ascending=True, na_position='last').sort_values(by=['waarde'], ascending=False).tail(15)

In [ ]:
df.describe()

In [ ]:
print(df.quantile(q=0.9))

In [ ]:
pd.pivot_table(df, values=['waarde'], index=['Periode', 'InstrumentSub'], aggfunc={'waarde': [min, max, len]}).tail(20)

In [ ]:
df_uitstaande = df[df['InstrumentSub']=='Uitstaande bedragen (mln euro\'s)'].copy()
df_uitstaande['lopende gemiddelde'] = df_uitstaande['waarde'].rolling(6, min_periods=4).mean()
df_uitstaande.head(12)

In [ ]:
df_a = df_uitstaande.copy()
df_a = df_a.set_index(['Instrument','InstrumentSub','Sector','Periode'],append=False)

df_b = df_uitstaande.copy()
df_b = df_b.set_index(['Instrument','InstrumentSub','Sector','Periode'],append=False)
df_b['waarde'] = df_b['lopende gemiddelde']

df_a.combine(df_b, np.minimum, fill_value = 8).head(10)

In [ ]:
df['Instrument'].value_counts()

In [ ]:
df.groupby(['Instrument','InstrumentSub']).agg({'waarde': [np.min, np.max, np.mean]})

In [ ]:
df.apply(pd.Series.unique, axis=0)

In [ ]:
df['waarde'].plot()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
df_a = df[df['Instrument']=='Woninghypotheken'].dropna()
df_a = df_a[df_a['InstrumentSub'].str.contains('Rente op uit')].set_index('Periode')

df_b = df[df['Instrument']=='Zakelijke kredietverlening'].dropna()
df_b = df_b[df_b['InstrumentSub'].str.contains('Rente op uit')].set_index('Periode')

df_c = pd.merge(df_b,df_a, how='inner', left_on=['Periode'], right_on=['Periode'], suffixes=('_c','_b'))
df_c.plot.scatter(x='waarde_c',y='waarde_b', alpha=0.5)

In [ ]:
df[df['Periode']=='2010-06'].plot.bar(y='waarde',x='InstrumentSub')

In [ ]:
df[df['Periode']=='2010-06'].plot(y='waarde',x='InstrumentSub', kind='barh')

### Data wegschrijven 


In [ ]:
df.to_pickle('results.pkl')

In [ ]:
df.to_csv('results.csv', index=False, decimal=',', sep='\t')

In [ ]:
with pd.ExcelWriter('results.xlsx') as writer:  
    df[df['Instrument']=='Woninghypotheken'].to_excel(writer, sheet_name='Woninghypotheken')
    df[df['Instrument']=='Zakelijke kredietverlening'].to_excel(writer, sheet_name='Zakelijke kredietverlening')